Setting up environment

In [1]:
!pip install hic-straw

from google.colab import drive
drive.mount('/content/drive')

# Importing necessary libraries
import tensorflow as tf # tensorflow 2.x
from tensorflow.keras import models, layers
from tensorflow.keras import backend as K
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Activation, Dropout, Lambda, Conv2DTranspose, AveragePooling2D
from keras.models import Model
from keras import backend as K
from matplotlib import pyplot as plt
import matplotlib
import time
import numpy as np
from scipy import stats
from keras import metrics
import random
import math
import hicstraw
import shutil
import os

  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.13.1-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.1-py3-none-any.whl (238 kB)
  Created wheel for hic-straw: filename=hic_straw-1.3.1-cp310-cp310-linux_x86_64.whl size=1616845 sha256=eee729fc1033effa7e46a01e8c06acde2c89755a626bfd4578c44f3748f98a8c
  Stored in directory: /root/.cache/pip/wheels/23/85/e0/14f1be833ddf1da34165e04938653e00b602eb93e834497ae4
Successfully built hic-straw
Mounted at /content/drive


In [2]:
# Setting up global variables
np.set_printoptions(suppress = True)
br = matplotlib.colors.LinearSegmentedColormap.from_list("bright_red",[(1,1,1),(1,0,0)])
max_distance = 100*1000*1000 # Maximum distance between two pieces on the same chromosome before unbouding the distance
WIDTH = 10 # Length/Width of each example
res = 1000*1000 # Resolution
ds_scale = 10 # Downsampling scale used when creating examples
# (e.g. examples will be created at 400 by 400 in 5 KB resolution then converted to 40 by 40 at 50 KB resolution)

Helper functions

In [3]:
# Helper functions used in data creation
# size = WIDTH
# partp is the partition point where the first region ends and the second region begins
# partp = len(region1) = size - len*region(2)

# Stitching a matrix together given the first region, second region, and common region

def construct_matrix(size, partp, matrix_chr1, matrix_chr2, matrix_comb):
  matrix = np.zeros([size, size])

  matrix[0:partp, 0:partp] = matrix_chr1
  matrix[partp:size, partp:size] = matrix_chr2

  common = matrix_comb

  matrix[0:partp, partp:size] = common
  matrix[partp:size, 0:partp] = np.flipud(np.rot90(common))

  return matrix

# Downsamples a matrix the same way Juicebox does
def hic_downsample(size, ds_scale, matrix):

  smatrix = np.zeros([int(size/ds_scale), int(size/ds_scale)])

  for i in range(int(size/ds_scale)):
    for j in range(int(size/ds_scale)):
      if i==j:
        smatrix[i, j] = np.sum(np.triu(matrix[ds_scale*i:ds_scale*(i+1), ds_scale*j:ds_scale*(j+1)]))
      else:
        smatrix[i, j] = np.sum(matrix[ds_scale*i:ds_scale*(i+1), ds_scale*j:ds_scale*(j+1)])

  return smatrix

# Determines the expected matrix and downsamples it
# observed/oe = observed / (observed/expected) = expected
def hic_expected_downsample(size, ds_scale, matrix_observed, matrix_oe):
  matrix = matrix_observed/matrix_oe
  return hic_downsample(size, ds_scale, matrix)

# Grabbing the first, second, and common chromosome regions given the start point of each region and the partition point (which gives the length of each region)
# chrom1 and chrom2 are indexes, not chromosome names
def grabFromStraw(hic, res, chrom1, chrom2, chr1_start, chr2_start, partp, size, data_type = 'observed', normalization = 'NONE', unit = 'BP'):
  chroms = hic.getChromosomes()

  matrix_chr1 = hic.getMatrixZoomData(chroms[chrom1].name, chroms[chrom1].name, data_type, normalization, unit, res)
  matrix_chr1 = matrix_chr1.getRecordsAsMatrix(chr1_start, chr1_start+(partp-1)*res, chr1_start, chr1_start+(partp-1)*res)

  matrix_chr2 = hic.getMatrixZoomData(chroms[chrom2].name, chroms[chrom2].name, data_type, normalization, unit, res)
  matrix_chr2 = matrix_chr2.getRecordsAsMatrix(chr2_start, chr2_start+(size-1-partp)*res, chr2_start, chr2_start+(size-1-partp)*res)

  matrix_comb = hic.getMatrixZoomData(chroms[chrom1].name, chroms[chrom2].name, data_type, normalization, unit, res)
  matrix_comb = matrix_comb.getRecordsAsMatrix(chr1_start, chr1_start+(partp-1)*res, chr2_start, chr2_start+(size-1-partp)*res)

  return matrix_chr1, matrix_chr2, matrix_comb

# distance_start and distance_end are the endpoints of the range determining the distance between the two regions when creating an example
# The actual distance between the regions is randomly selected from the range
# Checks if the length of a chromosome is greater than the size of the matrix + distance_end (necessary to create a proper example)
# If the chromosome cannot be used to create a proper example, the function returns False
def chromInvalid(hic, chrom, res, size, distance_start, distance_end):
  chroms = hic.getChromosomes()
  if distance_end == None: # If distance_end is unbounded then the chromosome only needs to be larger than size of matrix + distance_start
    distance_end = distance_start
  if chroms[chrom].length<res*size+distance_end:
    return True
  return False

# Creates a set of points that are used when creating an example
# Only diagonal points need to be included
# A point is (chromosome #, chrom_start_location)
def getPoints(chrom1, chrom2, chr1_start, chr2_start, res, size, partp):
  points = set()

  for i in range(partp):
    points.add((chrom1, chr1_start+i*res))
  for i in range(size-partp):
    points.add((chrom2, chr2_start+i*res))

  return points

# Another downsampling method which assigns the average of all the points in a block to the corresponding bin in the downsampled matrix
# Doesn't make much sense to use for hic data, would not recommend
def average(size, ds_scale, matrix):

  smatrix = np.zeros([int(size/ds_scale), int(size/ds_scale)])

  for i in range(int(size/ds_scale)):
    for j in range(int(size/ds_scale)):
      smatrix[i, j] = np.mean(matrix[ds_scale*i:ds_scale*(i+1), ds_scale*j:ds_scale*(j+1)])

  return smatrix

Creating training examples with interchromosomal breaks, intrachromosomal breaks, and no breaks

In [25]:
# Creating a folder to store the data in
folder = "Binary Classifier Training Data/"
if not os.path.exists(folder):
  os.mkdir(folder)

In [24]:
# Getting files

!wget https://www.dropbox.com/s/jqbb36osbzy121r/HIC11665_PPM_HiRise_rh_HiC.hic
!wget https://www.dropbox.com/s/p1l512m2lx1zkcf/HIC12835_PPM_HiRise_rh_HiC.hic
!wget https://www.dropbox.com/s/pxa547b3dsdd8qv/Daubentonia_madagascariensis_HiC.hic
!wget https://www.dropbox.com/s/1uq0ijiiuj39wbn/Chrysocyon_brachyurus_HiC.hic
!wget https://www.dropbox.com/s/otgsb1624ohru7s/Setonix_brachyurus_HiC.hic
# !wget https://www.dropbox.com/sh/3eh3lbzedwc0hrg/AADEdyqNcKklsw9ihtdxds0Ja/HIC16040_canFam3.hic
#!wget https://www.dropbox.com/s/qs5codrsewbfmf2/HIC16040_canFam3.hic - repeat file
# !wget https://www.dropbox.com/s/tb8b16zrg95996e/Cricetulus_griseus_HiC.hic
!wget https://www.dropbox.com/s/igegfumda9jrlxy/primary.hic
!wget https://www.dropbox.com/sh/3eh3lbzedwc0hrg/AABHmvN7KxvYvaQgbNwKceFha/HIC1024_deep.hic

files = ['HIC11665_PPM_HiRise_rh_HiC.hic',
         'HIC12835_PPM_HiRise_rh_HiC.hic',
         'Daubentonia_madagascariensis_HiC.hic',
         'Chrysocyon_brachyurus_HiC.hic',
         'Setonix_brachyurus_HiC.hic',
        # 'HIC16040_canFam3.hic',
         'primary.hic',
         'HIC1024_deep.hic']

--2024-08-08 15:57:02--  https://www.dropbox.com/s/p1l512m2lx1zkcf/HIC12835_PPM_HiRise_rh_HiC.hic
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/p1l512m2lx1zkcf/HIC12835_PPM_HiRise_rh_HiC.hic [following]
--2024-08-08 15:57:03--  https://www.dropbox.com/s/raw/p1l512m2lx1zkcf/HIC12835_PPM_HiRise_rh_HiC.hic
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc81ac454ac26e38a5032809d35a.dl.dropboxusercontent.com/cd/0/inline/CYQNZ4QmPP-o7PZcMuAKsgsFWhEbgg4YSxpIi8j2KgxJSlWuk65i2-pSKKGHZ2uvOQ9jVEo9lsDnIciJvAg4AkOhcEb5Bo6DF0UAR3MY8DT1rj4IWlGMdUNlgRreV5GoyNuTdax3m4qbn8QH0huQWIZy/file# [following]
--2024-08-08 15:57:04--  https://uc81ac454ac26e38a5032809d35a.dl.dropboxusercontent.com/cd/0/inline/CYQNZ4QmPP-o7PZcMuAKsgsFWhEbgg4YSxpIi8j2KgxJSl

In [35]:
# Creates interchromsomal breaks
# used_points is the set of all points that have already been used to ensure the same point is not used in multiple examples
# rand_partp=True sets the partition point to anywhere in the matrix except the middle where the model should detect the break
# perfect_break=True sets the partition point to the middle of the matrix to create a clean break
# one_bin_over=True sets the partition point to one bin away from the middle of the matrix (e.g. if the matrix is size 10, the partition point will be in bin 4 or 6)
# At most, only one of rand_partp, perfect_break, and one_bin_over should be set to true at any time
# If the break is not in the middle, the distance between regions is considered to be 0
def data_interchromosomal(hic, size=40, res = 100000, ds_scale=10, used_points=set(), rand_partp=False, perfect_break=False, one_bin_over=False):

  size *= ds_scale # Creating larger matrix to downsample later

  # If res is not a multiple of ds_scale downsampling won't work properly
  if not res%ds_scale == 0:
    print("ds_scale should be a divsor of res")
    return

  res //= ds_scale # Creating at lower resolution to downsample later

  distance = np.inf # distance between two separate chromosomes

  partp = size//2 - int(random.random()*ds_scale) # Partition point should be in the last bin of the first half of the downsampled matrix (e.g. 5th bin in 10 by 10 matrix)

  if rand_partp:
    distance = 0 # distance is 0 in the region the model should look at
    while (partp<=size//2 and partp>size//2-ds_scale):
      partp = int(random.random()*(size-1))+1

  elif perfect_break:
    partp = size//2 # Creates a clean break

  elif one_bin_over: # distance is 0 in the region the model should look at
    where = int(random.random()*2)
    if where==0:
      partp = size//2 + int(random.random()*ds_scale+1) # Partition point will be one bin to the right
    else:
      partp = size//2 - ds_scale - int(random.random()*ds_scale) # Partition point will be one bin to the left

  chroms = hic.getChromosomes()
  chrom1 = int(random.random()*(len(chroms)-1))+1 # Selecting a chromosome by getting its index in chroms
  # Selecting 0 as an index is invalid because chroms[0] is All

  # Selecting a second, different chromosome
  chrom2 = chrom1
  while(chrom2==chrom1):
    chrom2 = int(random.random()*(len(chroms)-1))+1

  # Checking if both chromosomes are larger than the regions that will be drawn from them
  if (chroms[chrom1].length<res*partp or chroms[chrom2].length<(size-partp)*res):
    # Rerunning the function if the answer is no
    return data_interchromosomal(hic, size//ds_scale, res*ds_scale, ds_scale, used_points, rand_partp, perfect_break, one_bin_over)

  # Getting a random start location (at a multiple of res) so the full region will fit in between the start location and end of the chromosome
  chr1_start = int(random.random()*(chroms[chrom1].length//res-partp+1))*res
  chr2_start = int(random.random()*(chroms[chrom2].length//res-size+1+partp))*res

  # Getting points that will be used
  points = getPoints(chrom1, chrom2, chr1_start, chr2_start, res, size, partp)

  # If there is overlap between points that will be used and points that have been used, the function is rerun
  if not len(used_points&points) == 0:
    return data_interchromosomal(hic, size//ds_scale, res*ds_scale, ds_scale, used_points, rand_partp, perfect_break, one_bin_over)

  used_points.update(points)

  # Grabbing and constructing observed matrix
  matrix_chr1, matrix_chr2, matrix_comb = grabFromStraw(hic, res, chrom1, chrom2, chr1_start, chr2_start, partp, size, 'observed', 'NONE', 'BP')
  matrix = construct_matrix(size, partp, matrix_chr1, matrix_chr2, matrix_comb)

  # Grabbing and constructing oe matrix
  matrix_chr1, matrix_chr2, matrix_comb = grabFromStraw(hic, res, chrom1, chrom2, chr1_start, chr2_start, partp, size, 'oe', 'NONE', 'BP')
  matrix2 = construct_matrix(size, partp, matrix_chr1, matrix_chr2, matrix_comb)

  # Downsampling
  observed_matrix = hic_downsample(size, ds_scale, matrix)
  expected_matrix = hic_expected_downsample(size, ds_scale, matrix, matrix2)

  return observed_matrix, observed_matrix/expected_matrix, distance, used_points

In [6]:
# Creates an intrachromosomal break with the distance in between regions belonging to a specific range
# distance_start and distance_end are the endpoints of the range
# If distance_end=None then the range is unbounded
def data_specific_range_intrachromosomal(hic, size=40, res = 100000, ds_scale=10, distance_start=0, distance_end=None, used_points = set(), rand_partp=False, perfect_break=False, one_bin_over=False):

  size *= ds_scale
  if not res%ds_scale == 0:
    print("ds_scale should be a divsor of res")
    return
  res //= ds_scale

  distance = np.inf

  partp = size//2 - int(random.random()*ds_scale) # Partition point should be in the last bin of the first half of the downsampled matrix (e.g. 5th bin in 10 by 10 matrix)

  if rand_partp:
    distance = 0
    while (partp<=size//2 and partp>size//2-ds_scale):
      partp = int(random.random()*(size-1))+1

  elif perfect_break:
    partp = size//2 # Creates a clean break

  elif one_bin_over:
    distance=0
    where = int(random.random()*2)
    if where==0:
      partp = size//2 + int(random.random()*ds_scale+1) # Partition point will be one bin to the right
    else:
      partp = size//2 - ds_scale - int(random.random()*ds_scale) # Partition point will be one bin to the left

  chroms = hic.getChromosomes()
  chrom = int(random.random()*(len(chroms)-1))+1

  # Checks to see if chromosome is large enough to create example
  if chromInvalid(hic, chrom, res, size, distance_start, distance_end):
    return data_specific_range_intrachromosomal(hic, size//ds_scale, res*ds_scale, ds_scale, distance_start, distance_end, used_points, rand_partp, perfect_break, one_bin_over)

  # Choosing a random distance (that is a multiple of res) from the range
  if distance_end == None:
    distance = int(random.random()*(chroms[chrom].length//res-distance_start//res))*res + distance_start
  else:
    distance = int((distance_end-distance_start)//res * random.random())*res + distance_start

  # Getting a random start location (at a multiple of res) so both regions and the distance in between them will fit in between the start location and end of the chromosome
  chr_start = int(random.random()*(chroms[chrom].length//res-size-distance//res)) * res

  # Starting point of the second region
  chr_continue = chr_start + distance + partp*res

  points = getPoints(chrom, chrom, chr_start, chr_continue, res, size, partp)

  if not len(used_points&points) == 0:
    return data_specific_range_intrachromosomal(hic, size//ds_scale, res*ds_scale, ds_scale, distance_start, distance_end, used_points, rand_partp, perfect_break, one_bin_over)

  used_points.update(points)

  matrix_chr1, matrix_chr2, matrix_comb = grabFromStraw(hic, res, chrom, chrom, chr_start, chr_continue, partp, size, 'observed', 'NONE', 'BP')
  matrix = construct_matrix(size, partp, matrix_chr1, matrix_chr2, matrix_comb)

  matrix_chr1, matrix_chr2, matrix_comb = grabFromStraw(hic, res, chrom, chrom, chr_start, chr_continue, partp, size, 'oe', 'NONE', 'BP')
  matrix2 = construct_matrix(size, partp, matrix_chr1, matrix_chr2, matrix_comb)

  observed_matrix = hic_downsample(size, ds_scale, matrix)
  expected_matrix = hic_expected_downsample(size, ds_scale, matrix, matrix2)

  return observed_matrix, observed_matrix/expected_matrix, distance, used_points

In [14]:
# Draws continuous regions
def data_continuous(hic, size=40, res = 100000, ds_scale=10, used_points = set()):
  size *= ds_scale
  if not res%ds_scale == 0:
    print("ds_scale should be a divsor of res")
    return
  res //= ds_scale

  chroms = hic.getChromosomes()
  chrom = int(random.random()*(len(chroms)-1))+1

  if (chroms[chrom].length<res*size):
    return data_continuous(hic, size//ds_scale, res*ds_scale, ds_scale, used_points)

  chr_start = int(random.random()*(chroms[chrom].length//res-(size-1)))
  points = set()

  for i in range(size):
    points.add((chrom, chr_start+i*res))

  if len(used_points&points) != 0:
    return data_continuous(hic, size//ds_scale, res*ds_scale, ds_scale, used_points)

  used_points.update(points)

  matrix_chr1 = hic.getMatrixZoomData(chroms[chrom].name, chroms[chrom].name, "observed", "NONE", "BP", res)
  matrix = matrix_chr1.getRecordsAsMatrix(chr_start, chr_start+(size-1)*res, chr_start, chr_start+(size-1)*res)

  matrix_chr2 = hic.getMatrixZoomData(chroms[chrom].name, chroms[chrom].name, "oe", "NONE", "BP", res)
  matrix2 = matrix_chr2.getRecordsAsMatrix(chr_start, chr_start+(size-1)*res, chr_start, chr_start+(size-1)*res)

  observed_matrix = hic_downsample(size, ds_scale, matrix)
  expected_matrix = hic_expected_downsample(size, ds_scale, matrix, matrix2)

  # The distance between the pieces is 0
  return observed_matrix, observed_matrix/expected_matrix, 0, used_points

In [ ]:
# Creating training data

training_examples = [100, 100, 100, 100, 100] # Number of training examples for each type of data

for idx, file in enumerate(files):
  hic = hicstraw.HiCFile(file)
  chroms = hic.getChromosomes()
  # Creating a subfolder for the specific hic file
  if not os.path.exists(folder + file):
    os.mkdir(folder + file)

  # If the file already exists, skip it
  for bin_num in range(len(training_examples)):
    file_name_data = "Type" + str(bin_num+1) + "_data.npy"
    file_name_label = "Type" + str(bin_num+1) + "_label.npy"
    if os.path.isfile(folder+file+"/"+file_name_data):
      print("File already exists", folder, file, file_name_data)
      continue

    training_data = np.zeros([training_examples[bin_num], WIDTH, WIDTH, 2])
    training_label = np.zeros([training_examples[bin_num]])
    used_points = set()

    k=0

    try:
      while (k<training_examples[bin_num]):
        # Generating training examples
        if bin_num==0:
          eData1, eData2, label, used_points = data_interchromosomal(hic, size=WIDTH, res = res, ds_scale=ds_scale, used_points=used_points, rand_partp=False, perfect_break=False, one_bin_over=False)
        elif bin_num==1:
          eData1, eData2, label, used_points = data_interchromosomal(hic, size=WIDTH, res = res, ds_scale=ds_scale, used_points=used_points, rand_partp=False, perfect_break=True, one_bin_over=False)
        elif bin_num==2:
          eData1, eData2, label, used_points = data_continuous(hic, size=WIDTH, res = res, ds_scale=ds_scale, used_points = used_points)
        elif bin_num==3:
          eData1, eData2, label, used_points = data_interchromosomal(hic, size=WIDTH, res = res, ds_scale=ds_scale, used_points=used_points, rand_partp=False, perfect_break=False, one_bin_over=True)
        elif bin_num==4:
          eData1, eData2, label, used_points = data_interchromosomal(hic, size=WIDTH, res = res, ds_scale=ds_scale, used_points=used_points, rand_partp=True, perfect_break=False, one_bin_over=False)
        # elif bin_num==5:
        #   data_specific_range_intrachromosomal(hic, size=WIDTH, res = res, ds_scale=ds_scale, distance_start=5000000, distance_end=None, used_points = used_points, rand_partp=False, perfect_break=False, one_bin_over=False)

        training_data[k, :, :, 0] = eData1 # Observed data
        training_data[k, :, :, 1] = eData2 # OE data
        training_label[k] = label

        k+=1
    except Exception as e:
      # If there are too few examples that can be created from the given data, this exception will be thrown
      print("Not enough examples for", file, file_name_data)

    # Saving training data and moving it into the correct folder
    np.save(file_name_data, training_data)
    shutil.move(file_name_data, folder+file)
    np.save(file_name_label, training_label)
    shutil.move(file_name_label, folder+file)